## Einleitung

### Use-Case

### Github Repo

## Modellarchitektur (Jan)

## Datengrundlage (Sowohl EDA wie auch beschreiben (newsqa usw.))

## Erroranalyse (min. 3 Stück) (Florin)

Evaluation Small -> welche Fehler macht das Modell?

## Verbesserungvorschläge und dazugehörige Evaluation

## Erklärung

## Weitere Ideen

## Fazit